<a href="https://www.kaggle.com/code/karinae/trainmodel-pnn-scg?scriptVersionId=237693068" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import warnings
warnings.simplefilter('ignore', FutureWarning)

In [ ]:
!pip install scikit-learn==1.2.2 imbalanced-learn==0.10.1

# Data đã xử lý

In [ ]:
df_april = pd.read_csv('/kaggle/input/clean-aws-month/final_data/filled_data_april.csv')
df_october = pd.read_csv('/kaggle/input/clean-aws-month/final_data/filled_data_october.csv')

In [ ]:
df_april.head()

# Feature đã chọn dựa trên phương án 1
- T4:  CAPE, KX, R500, R850, TCLW, TCW, U250, U850, V250, V850, B10B, B11B, B14B, I2B, I4B, IRB, WVB
- T10:  EWSS, KX, PEV, R250, R500, R850, SSHF, TCLW, TCW, U250, U850, V250, V850, B11B,  B14B, I4B, IRB

In [ ]:
features_april = ['CAPE', 'KX', 'R500', 'R850', 'TCLW', 'TCW', 'U250', 'U850', 'V250', 'V850', 'B10B', 'B11B', 'B14B', 'I2B', 'I4B', 'IRB', 'WVB']
features_october = ['EWSS', 'KX', 'PEV', 'R250', 'R500', 'R850', 'SSHF', 'TCLW', 'TCW', 'U250', 'U850', 'V250', 'V850', 'B11B', 'B14B', 'I4B', 'IRB']

# Chia train test theo thời gian

In [ ]:
def check_missing_hours(df, month):
    df['datetime'] = pd.to_datetime(df['datetime']) 
    df = df[df['datetime'].dt.month == month].copy()

    all_dates = df['datetime'].dt.date.unique()
    all_expected_hours = []
    for date in all_dates:
        hours = pd.date_range(f"{date} 00:00:00", f"{date} 23:00:00", freq="H")
        all_expected_hours.extend(hours)
    
    expected_df = pd.DataFrame(all_expected_hours, columns=["datetime"])
    missing_times = expected_df[~expected_df['datetime'].isin(df['datetime'])]
    return missing_times

missing_april = check_missing_hours(df_april, 4)
missing_october = check_missing_hours(df_october, 10)

print("T4:")
print(missing_april['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S').to_string(index=False))

print("T10:")
print(missing_october['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S').to_string(index=False))


In [ ]:
def create_missing_hours_heatmap(missing_times, month_name):
    missing_times['Year'] = missing_times['datetime'].dt.year
    missing_times['Month'] = missing_times['datetime'].dt.month
    missing_times['Day'] = missing_times['datetime'].dt.day
    missing_times['Hour'] = missing_times['datetime'].dt.hour
    
    missing_counts = missing_times.groupby(['Year', 'Month', 'Day']).size().reset_index(name='Missing_Hours')

    pivot_data = missing_counts.pivot_table(
        index=['Year', 'Month'], columns='Day', values='Missing_Hours', fill_value=0
    )
    
    pivot_data.index = [f'{year} - Tháng {month}' for year, month in pivot_data.index]

    plt.figure(figsize=(16, 8))
    sns.heatmap(pivot_data, cmap='YlGnBu', annot=True, fmt='g', linewidths=0.5, square=True, vmin=0, vmax=24)
    plt.ylabel('Năm - Tháng', fontsize=12)
    plt.show()

create_missing_hours_heatmap(missing_october, "Tháng 10")

- Tháng 4 thiếu giờ ở ngày 2020-04-06 và 2020-04-13
- Tháng 10 thiếu giờ chủ yếu ở năm 2019 trong ngày 7,8,9,19,15-21,23,26,27,29,30

  => quyết định chia tập train từ 4/4/2019 đến 28/4/2019 và 4/4/2020 đến 28/4/2020, tập test là những ngày còn lại
 -  tương tự cho tháng 10

In [ ]:
def split_data_by_multiple_ranges(df, train_ranges):
    train_mask = False
    for start, end in train_ranges:
        train_mask |= (df['datetime'] >= start) & (df['datetime'] < end)
    train_df = df[train_mask]
    test_df = df[~train_mask]
    return train_df, test_df

def convert_rain_label(df):
    df['AWS'] = df['AWS'].apply(lambda x: 1 if x > 0 else 0)
    return df

df_april = convert_rain_label(df_april)
df_october = convert_rain_label(df_october)

train_ranges_april = [("2019-04-04", "2019-04-29"), ("2020-04-04", "2020-04-29")]
train_ranges_october = [("2019-10-04", "2019-10-29"), ("2020-10-04", "2020-10-29")]

train_april, test_april = split_data_by_multiple_ranges(df_april, train_ranges_april)
train_october, test_october = split_data_by_multiple_ranges(df_october, train_ranges_october)

print(f"Tháng 4 - Train: {train_april.shape}")
print(f"Tháng 4 - Test: {test_april.shape}")
print(f"Tháng 10 - Train: {train_october.shape}")
print(f"Tháng 10 - Test: {test_october.shape}")


In [ ]:
rain_april = train_april[train_april['AWS'] == 1].shape[0]
no_rain_april = train_april[train_april['AWS'] == 0].shape[0]
total_april = train_april.shape[0]
rain_ratio_april = rain_april / total_april
no_rain_ratio_april = no_rain_april / total_april

rain_october = train_october[train_october['AWS'] == 1].shape[0]
no_rain_october = train_october[train_october['AWS'] == 0].shape[0]
total_october = train_october.shape[0]
rain_ratio_october = rain_october / total_october
no_rain_ratio_october = no_rain_october / total_october

labels = ['Mưa', 'Không mưa']
rain_data_april = [rain_ratio_april, no_rain_ratio_april]
rain_data_october = [rain_ratio_october, no_rain_ratio_october]

fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].pie(rain_data_april, labels=labels, autopct='%1.2f%%', startangle=90, colors=['#1f77b4', '#ff7f0e'])
axes[0].set_title(f"Tỷ lệ mưa và không mưa - Tháng 4")

axes[1].pie(rain_data_october, labels=labels, autopct='%1.2f%%', startangle=90, colors=['#1f77b4', '#ff7f0e'])
axes[1].set_title(f"Tỷ lệ mưa và không mưa - Tháng 10")

plt.tight_layout()
plt.show()

# Min-max normalization

In [ ]:
train_april.shape[1]

In [ ]:
X_train_april = train_april[features_april]
y_train_april = train_april['AWS']

X_test_april = test_april[features_april]
y_test_april = test_april['AWS']

X_train_october = train_october[features_october]
y_train_october = train_october['AWS']

X_test_october = test_october[features_october]
y_test_october = test_october['AWS']

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler_april = MinMaxScaler()
X_train_april_scaled = scaler_april.fit_transform(X_train_april)
X_test_april_scaled = scaler_april.transform(X_test_april)

scaler_october = MinMaxScaler()
X_train_october_scaled = scaler_october.fit_transform(X_train_october)
X_test_october_scaled = scaler_october.transform(X_test_october)

In [ ]:
X_train_april_scaled = pd.DataFrame(X_train_april_scaled, columns=X_train_april.columns)
X_test_april_scaled = pd.DataFrame(X_test_april_scaled, columns=X_test_april.columns)

X_train_october_scaled = pd.DataFrame(X_train_october_scaled, columns=X_train_october.columns)
X_test_october_scaled = pd.DataFrame(X_test_october_scaled, columns=X_test_october.columns)

# Train lúc chưa cân bằng

In [ ]:
def train_and_evaluate_multiple_models(models_info, model_class, **model_params):
    trained_models = {}
    predictions = {}
    
    for name, X_train, y_train, X_test, y_test in models_info:
        print(f"\n{name}...")
        
        # Khởi tạo mô hình với các tham số đầu vào
        model = model_class(**model_params)
        model.fit(X_train, y_train)
        trained_models[name] = model
        
        # Dự đoán trên tập test
        predictions[name] = model.predict(X_test)

    # Đánh giá mô hình
    for name, X_train, y_train, X_test, y_test in models_info:
        print(f"\n{name}:")
        y_pred = predictions[name]
        
        # In classification report
        print(classification_report(y_test, y_pred, target_names=['No Rain', 'Rain']))
        
        # Hiển thị confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        cm_display = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['No Rain', 'Rain'])
        cm_display.plot(cmap=plt.cm.Blues)
        plt.title(f"Confusion Matrix - {name}")
        plt.show()

In [ ]:
models_info = [
    ("Tháng 4", X_train_april_scaled, y_train_april, X_test_april_scaled, y_test_april),
    ("Tháng 10", X_train_october_scaled, y_train_october, X_test_october_scaled, y_test_october)
]

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt  

In [ ]:
model_params = {
    'n_estimators': 100,
    'random_state': 42
}

In [ ]:
train_and_evaluate_multiple_models(models_info, RandomForestClassifier, **model_params)

Tháng 4:
              precision    recall  f1-score   support

     No Rain       0.90      0.99      0.95     46790
        Rain       0.52      0.06      0.11      5289

    accuracy                           0.90     52079
    macro avg       0.71      0.53      0.53     52079
    weighted avg       0.86      0.90      0.86     52079


Tháng 10:
              precision    recall  f1-score   support

     No Rain       0.89      0.96      0.92     57109
        Rain       0.73      0.50      0.59     13583

    accuracy                           0.87     70692
    macro avg       0.81      0.73      0.76     70692
    weighted avg       0.86      0.87      0.86     70692




## XGboost

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgboost_params = {
    'n_estimators': 100,
    'random_state': 42
}

In [ ]:
train_and_evaluate_multiple_models(models_info, XGBClassifier, **xgboost_params)

Tháng 4:
              precision    recall  f1-score   support

     No Rain       0.91      0.98      0.94     46790
        Rain       0.39      0.10      0.16      5289

    accuracy                           0.89     52079
    macro avg       0.65      0.54      0.55     52079
    weighted avg       0.85      0.89      0.86     52079


Tháng 10:
              precision    recall  f1-score   support

     No Rain       0.90      0.94      0.92     57109
        Rain       0.71      0.58      0.64     13583

    accuracy                           0.87     70692
    macro avg       0.81      0.76      0.78     70692
    weighted avg       0.87      0.87      0.87     70692

## SVM

In [ ]:
from sklearn.svm import SVC

In [ ]:
svm_params = {
    'kernel': 'rbf',       
    'C': 1.0,
    'gamma': 'scale',       
    'probability': True,     
    'random_state': 42
}

In [ ]:
train_and_evaluate_multiple_models(models_info, SVC, **svm_params)

## PNN

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.metrics import pairwise_distances

class RBFLayer:
    def __init__(self, centers, sigma=1.0):
        self.centers = centers
        self.sigma = sigma

    def compute(self, X):
        diff = X[:, np.newaxis, :] - self.centers
        dist_sq = np.sum(diff**2, axis=2)
        return np.exp(-dist_sq / (2 * self.sigma**2))

def softmax(z):
    # z: (n_samples, n_classes)
    z_max = np.max(z, axis=1, keepdims=True)
    e_z = np.exp(z - z_max)
    return e_z / e_z.sum(axis=1, keepdims=True)


#Define PNN Model
class PNN:
    def __init__(self, sigma=1.0, batch_size=1000):
        self.sigma = sigma
        self.centers = None
        self.batch_size = batch_size

    def fit(self, X_train, y_train):
        if isinstance(X_train, pd.DataFrame):
            X_train = X_train.values
        else:
            X_train = X_train
        if isinstance(y_train, pd.Series):
            y_train = y_train.values
        else:
            y_train = y_train

        classes = np.unique(y_train)
        centers = []
        for c in classes:
            centers.append(X_train[y_train == c].mean(axis=0))
        self.centers = np.vstack(centers)
        self.rbf = RBFLayer(self.centers, sigma=self.sigma)

    def predict_proba(self, X):
        if isinstance(X, pd.DataFrame):
            X = X.values
        else:
            X = X
            
        n = X.shape[0]
        if self.batch_size is None:
            raw = self.rbf.compute(X)
            return softmax(raw)
        # batch processing
        probs = []
        for start in range(0, n, self.batch_size):
            end = min(start + self.batch_size, n)
            raw = self.rbf.compute(X[start:end])
            probs.append(softmax(raw))
        return np.vstack(probs)

    def predict(self, X):
        # trả về nhãn 0 hoặc 1
        probs = self.predict_proba(X)
        return np.argmax(probs, axis=1)

In [ ]:
#Fine-tune + train
sigmas = np.logspace(-2, 1, 50)
param_grid = {'sigma': sigmas.tolist()}  # Grid search for sigma values

best_score = 0
best_sigma = 0

# Perform grid search over the values of sigma
print("Fine tuning...")
for name, train_X, train_Y, X_test, y_test in models_info:
    X_train, X_val, y_train, y_val = train_test_split(
        train_X, train_Y, test_size=0.2, random_state=42
    )
    print(f"{name}...")
    for sigma in param_grid['sigma']:
        pnn_batch = PNN(sigma=sigma, batch_size=2048)
        pnn_batch.fit(X_train, y_train)
        y_pred = pnn_batch.predict(X_val)
        score = classification_report(y_val, y_pred, zero_division=0, output_dict=True)['macro avg']['f1-score']
    
        if score > best_score:
            best_score = score
            best_sigma = sigma

    print(f"Best sigma: {best_sigma} with f1: {best_score}")

# Final evaluation with the best sigma
print("Train with best param...")
for name, X_train, y_train, X_test, y_test in models_info:
    print(f"{name}...")
    print(X_train.shape)
    pnn_batch = PNN(sigma=best_sigma, batch_size=1000)
    pnn_batch.fit(X_train, y_train)
    y_pred = pnn_batch.predict(X_test)

    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    # Optionally, plot the confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["No Rain", "Rain"], yticklabels=["No Rain", "Rain"])
    plt.title("Confusion Matrix")
    plt.show()



## SCG

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt

In [ ]:
def softmax(Z):
    # Ensure Z is a numpy array of shape (n_samples, n_classes)
    Z_arr = Z.values if hasattr(Z, "values") else np.array(Z)
    # 1) subtract the max for numerical stability
    #    max_per_row: shape (n_samples,)
    max_per_row = np.max(Z_arr, axis=1)
    #    reshape to (n_samples, 1) so broadcasting works
    max_per_row = max_per_row.reshape(-1, 1)
    # 2) exponentiate
    expZ = np.exp(Z_arr - max_per_row)
    # 3) sum per row, then reshape to (n_samples, 1)
    sum_per_row = np.sum(expZ, axis=1).reshape(-1, 1)
    # 4) divide
    return expZ / sum_per_row

def pack(W1, b1, W2, b2):
    """
    Flatten and concatenate parameters, handling pandas DataFrame/Series inputs.
    """
    # Convert DataFrame/Series to numpy arrays
    W1_arr = W1.values if isinstance(W1, pd.DataFrame) else np.array(W1)
    b1_arr = b1.values if isinstance(b1, (pd.Series, pd.DataFrame)) else np.array(b1)
    W2_arr = W2.values if isinstance(W2, pd.DataFrame) else np.array(W2)
    b2_arr = b2.values if isinstance(b2, (pd.Series, pd.DataFrame)) else np.array(b2)
    # Flatten and concatenate
    return np.hstack([
        W1_arr.ravel(),
        b1_arr.ravel(),
        W2_arr.ravel(),
        b2_arr.ravel()
    ])

def unpack(p, input_dim, hidden_dim, output_dim):
    idx = 0
    W1 = p[idx:idx + hidden_dim*input_dim].reshape(hidden_dim, input_dim)
    idx += hidden_dim*input_dim
    b1 = p[idx:idx + hidden_dim]
    idx += hidden_dim
    W2 = p[idx:idx + output_dim*hidden_dim].reshape(output_dim, hidden_dim)
    idx += output_dim*hidden_dim
    b2 = p[idx:idx + output_dim]
    return W1, b1, W2, b2

def loss_and_grad(p, X_train, Y_train_oh, input_dim, hidden_dim, output_dim):
    W1, b1, W2, b2 = unpack(p, input_dim, hidden_dim, output_dim)
    m = X_train.shape[0]
    Z1 = X_train.dot(W1.T) + b1
    A1 = np.tanh(Z1)
    Z2 = A1.dot(W2.T) + b2
    A2 = softmax(Z2)
    loss = -np.mean(np.sum(Y_train_oh * np.log(A2 + 1e-12), axis=1))
    dZ2 = (A2 - Y_train_oh) / m
    dW2 = dZ2.T.dot(A1)
    db2 = dZ2.sum(axis=0)
    dA1 = dZ2.dot(W2)
    dZ1 = dA1 * (1 - np.tanh(Z1)**2)
    dW1 = dZ1.T.dot(X_train)
    db1 = dZ1.sum(axis=0)
    return loss, pack(dW1, db1, dW2, db2)

def trainscg(loss_grad_func, x0, epochs=8, **lg_kwargs):
    sigma0, lambd = 1e-6, 1e-6
    x = x0.copy()
    f, g = loss_grad_func(x, **lg_kwargs)
    d = -g
    errors = [f]
    for epoch in range(epochs):
        mu = d.dot(d)
        sigma = sigma0 / np.sqrt(mu) if mu > 0 else sigma0
        _, g1 = loss_grad_func(x + sigma*d, **lg_kwargs)
        s = (g1 - g) / sigma
        delta = d.dot(s)
        if delta <= 0:
            delta = lambd * mu
            lambd -= delta / mu
        alpha = -d.dot(g) / (delta + lambd*mu)
        x_new = x + alpha*d
        f_new, g_new = loss_grad_func(x_new, **lg_kwargs)
        Delta = 2*(f - f_new) / (alpha * d.dot(g))
        if Delta >= 0:
            x, f, g = x_new, f_new, g_new
            lambd *= max(1/3, 1 - (2*Delta - 1)**3)
            beta = (g.dot(g) - g.dot(g_new)) / (d.dot(g))
            d = -g + beta*d
        else:
            lambd += mu * (1 - Delta)
        errors.append(f)
        print(f"Epoch {epoch+1}/{epochs} — loss: {f:.6f}")
    return x, errors

# ——— Main loop per dataset ———
for name, X_train, y_train, X_test, y_test in models_info:
    print(f"\n=== Dataset: {name} ===")
    y_arr = y_train.to_numpy().reshape(-1, 1)
    encoder = OneHotEncoder(sparse=False, categories='auto')
    Y_train_oh = encoder.fit_transform(y_arr)
    
    input_dim  = X_train.shape[1]
    hidden_dim = 8
    output_dim = Y_train_oh.shape[1]  # should be 2
    
    # 3) Initialize parameters
    param_size  = hidden_dim*input_dim + hidden_dim + output_dim*hidden_dim + output_dim
    init_params = 0.01 * np.random.randn(param_size)
    
    # 4) Train with SCG
    opt_params, loss_history = trainscg(
        loss_and_grad,
        init_params,
        epochs=8,
        X_train=X_train,
        Y_train_oh=Y_train_oh,
        input_dim=input_dim,
        hidden_dim=hidden_dim,
        output_dim=output_dim
    )
    
    # 5) Unpack weights & biases
    W1_opt, b1_opt, W2_opt, b2_opt = unpack(opt_params, input_dim, hidden_dim, output_dim)
    
    # 6) Display performance curve
    plt.figure()
    plt.plot(range(len(loss_history)), loss_history, marker='o')
    plt.title(f"{name} – SCG loss curve")
    plt.xlabel("Epoch")
    plt.ylabel("Cross-entropy loss")
    plt.grid(True)
    plt.show()
    
    # 7) ROC & final eval
    def predict_proba(params, X):
        W1, b1, W2, b2 = unpack(params, input_dim, hidden_dim, output_dim)
        A1 = np.tanh(X.dot(W1.T) + b1)
        Z2 = A1.dot(W2.T) + b2
        return softmax(Z2)
    
    probs = predict_proba(opt_params, X_test)
    y_pred = np.argmax(probs, axis=1)
    
    # ROC
    fpr, tpr, _ = roc_curve(y_test, probs[:,1])
    roc_auc = auc(fpr, tpr)
    plt.figure()
    plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.2f}")
    plt.plot([0,1],[0,1],'k--')
    plt.title(f"{name} – ROC curve")
    plt.xlabel("False positive rate")
    plt.ylabel("True positive rate")
    plt.legend()
    plt.grid(True)
    plt.show()
    
    # Classification report + confusion matrix
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    cm = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix:\n", cm)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=encoder.categories_[0], yticklabels=encoder.categories_[0])
    plt.title(f"{name} – Confusion matrix")
    plt.show()

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score, classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns

# ——— Utility functions ———
def softmax(Z):
    # Ensure Z is a numpy array of shape (n_samples, n_classes)
    Z_arr = Z.values if hasattr(Z, "values") else np.array(Z)
    # 1) subtract the max for numerical stability
    #    max_per_row: shape (n_samples,)
    max_per_row = np.max(Z_arr, axis=1)
    #    reshape to (n_samples, 1) so broadcasting works
    max_per_row = max_per_row.reshape(-1, 1)
    # 2) exponentiate
    expZ = np.exp(Z_arr - max_per_row)
    # 3) sum per row, then reshape to (n_samples, 1)
    sum_per_row = np.sum(expZ, axis=1).reshape(-1, 1)
    # 4) divide
    return expZ / sum_per_row

def pack(W1, b1, W2, b2):
    """
    Flatten and concatenate parameters, handling pandas DataFrame/Series inputs.
    """
    # Convert DataFrame/Series to numpy arrays
    W1_arr = W1.values if isinstance(W1, pd.DataFrame) else np.array(W1)
    b1_arr = b1.values if isinstance(b1, (pd.Series, pd.DataFrame)) else np.array(b1)
    W2_arr = W2.values if isinstance(W2, pd.DataFrame) else np.array(W2)
    b2_arr = b2.values if isinstance(b2, (pd.Series, pd.DataFrame)) else np.array(b2)
    # Flatten and concatenate
    return np.hstack([
        W1_arr.ravel(),
        b1_arr.ravel(),
        W2_arr.ravel(),
        b2_arr.ravel()
    ])

def unpack(p, input_dim, hidden_dim, output_dim):
    idx = 0
    W1 = p[idx:idx + hidden_dim*input_dim].reshape(hidden_dim, input_dim)
    idx += hidden_dim*input_dim
    b1 = p[idx:idx + hidden_dim]
    idx += hidden_dim
    W2 = p[idx:idx + output_dim*hidden_dim].reshape(output_dim, hidden_dim)
    idx += output_dim*hidden_dim
    b2 = p[idx:idx + output_dim]
    return W1, b1, W2, b2

def trainscg(loss_grad_func, x0, epochs=8, **lg_kwargs):
    sigma0, lambd = 1e-6, 1e-6
    x = x0.copy()
    f, g = loss_grad_func(x, **lg_kwargs)
    d = -g
    errors = [f]
    for epoch in range(epochs):
        mu = d.dot(d)
        sigma = sigma0 / np.sqrt(mu) if mu > 0 else sigma0
        _, g1 = loss_grad_func(x + sigma*d, **lg_kwargs)
        s = (g1 - g) / sigma
        delta = d.dot(s)
        if delta <= 0:
            delta = lambd * mu
            lambd -= delta / mu
        alpha = -d.dot(g) / (delta + lambd*mu)
        x_new = x + alpha*d
        f_new, g_new = loss_grad_func(x_new, **lg_kwargs)
        Delta = 2*(f - f_new) / (alpha * d.dot(g))
        if Delta >= 0:
            x, f, g = x_new, f_new, g_new
            lambd *= max(1/3, 1 - (2*Delta - 1)**3)
            beta = (g.dot(g) - g.dot(g_new)) / (d.dot(g))
            d = -g + beta*d
        else:
            lambd += mu * (1 - Delta)
        errors.append(f)
    return x, errors

def loss_grad_factory(X, Y_oh, input_dim, hidden_dim, output_dim):
    def loss_and_grad(p):
        W1, b1, W2, b2 = unpack(p, input_dim, hidden_dim, output_dim)
        m = X.shape[0]
        Z1 = X.dot(W1.T) + b1
        A1 = np.tanh(Z1)
        Z2 = A1.dot(W2.T) + b2
        A2 = softmax(Z2)
        loss = -np.mean(np.sum(Y_oh * np.log(A2 + 1e-12), axis=1))
        dZ2 = (A2 - Y_oh) / m
        dW2 = dZ2.T.dot(A1)
        db2 = dZ2.sum(axis=0)
        dA1 = dZ2.dot(W2)
        dZ1 = dA1 * (1 - np.tanh(Z1)**2)
        dW1 = dZ1.T.dot(X)
        db1 = dZ1.sum(axis=0)
        return loss, pack(dW1, db1, dW2, db2)
    return loss_and_grad

# ——— Main grid search & final train ———
param_grid = {'hidden_dim': [10, 20, 50, 100, 150, 17, 25, 34, 50, 68, 85]}

for name, X_trval, y_trval, X_test, y_test in models_info:
    print(f"\n=== Dataset: {name} ===")
    # split train+val from test
    X_train, X_val, y_train, y_val = train_test_split(
        X_trval, y_trval, test_size=0.2, random_state=42
    )
    # one-hot encode y_train
    y_arr = y_train.to_numpy().reshape(-1, 1)
    encoder = OneHotEncoder(sparse=False, categories='auto')
    Y_train_oh = encoder.fit_transform(y_arr.reshape(-1,1))
    y_arv = y_val.to_numpy().reshape(-1, 1)
    Y_val_oh   = encoder.transform(y_arv.reshape(-1,1))
    # final test encoding not needed for training
    
    # scale assumed done before, X_trval and X_test are already scaled
    
    input_dim  = X_train.shape[1]
    output_dim = Y_train_oh.shape[1]
    
    best_h, best_loss = None, np.inf
    for h in param_grid['hidden_dim']:
        # init params
        param_size  = h*input_dim + h + output_dim*h + output_dim
        init_p = 0.01 * np.random.randn(param_size)
        # train SCG
        lg = loss_grad_factory(X_train, Y_train_oh, input_dim, h, output_dim)
        opt_p, _ = trainscg(lg, init_p, epochs=8)
        # predict on val
        W1, b1, W2, b2 = unpack(opt_p, input_dim, h, output_dim)
        Z1_val = X_val.dot(W1.T) + b1
        A1_val = np.tanh(Z1_val)
        Z2_val = A1_val.dot(W2.T) + b2
        A2_val = softmax(Z2_val)
        val_loss = -np.mean(np.sum(Y_val_oh * np.log(A2_val + 1e-12), axis=1))
        print(f" hidden_dim={h:<3} → val loss = {val_loss:.6f}")
        if val_loss < best_loss:
            best_loss, best_h = val_loss, h
    
    print(f"⇒ Best hidden_dim = {best_h}, validation loss = {best_loss:.6f}")
    
    # retrain on full train+val
    X_full = np.vstack([X_train, X_val])
    y_full = np.hstack([y_train, y_val])
    Y_full_oh = encoder.fit_transform(y_full.reshape(-1,1))
    param_size  = best_h*input_dim + best_h + output_dim*best_h + output_dim
    init_p_full = 0.01 * np.random.randn(param_size)
    lg_full = loss_grad_factory(X_full, Y_full_oh, input_dim, best_h, output_dim)
    opt_p_full, loss_hist = trainscg(lg_full, init_p_full, epochs=8)
    
    # evaluate on test
    W1f, b1f, W2f, b2f = unpack(opt_p_full, input_dim, best_h, output_dim)
    A1_test = np.tanh(X_test.dot(W1f.T) + b1f)
    probs_test = softmax(A1_test.dot(W2f.T) + b2f)
    preds_test = np.argmax(probs_test, axis=1)
    
    print("Test classification report:")
    print(classification_report(y_test, preds_test))
    cm = confusion_matrix(y_test, preds_test)
    print("Test confusion matrix:\n", cm)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title(f"{name} – hidden_dim={best_h}")
    plt.show()


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
import numpy as np

def train_and_evaluate_scg_ann(models_info, hidden_neurons=8, epochs=8, learning_rate=0.01):
    trained_models = {}
    predictions = {}

    for name, X_train, y_train, X_test, y_test in models_info:
        print(f"\n{name} - SCG-ANN...")

        # One-hot encode labels
        lb = LabelBinarizer()
        y_train_bin = lb.fit_transform(y_train)
        y_test_bin = lb.transform(y_test)

        model = Sequential()
        model.add(Dense(hidden_neurons, input_shape=(X_train.shape[1],), activation='tanh'))
        model.add(Dense(2, activation='softmax'))

        model.compile(optimizer=SGD(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
        model.fit(X_train, y_train_bin, epochs=epochs, verbose=0)

        y_pred_prob = model.predict(X_test)
        y_pred = np.argmax(y_pred_prob, axis=1)

        trained_models[name] = model
        predictions[name] = y_pred

    for (name, _, _, _, y_test) in models_info:
        y_pred = predictions[name]
        print(f"\n{name} - SCG-ANN:")
        print(classification_report(y_test, y_pred, target_names=['No Rain', 'Rain']))

        cm = confusion_matrix(y_test, y_pred)
        cm_display = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['No Rain', 'Rain'])
        cm_display.plot(cmap=plt.cm.Blues)
        plt.title(f"Confusion Matrix - SCG-ANN - {name}")
        plt.show()


# Cân bằng dữ liệu bằng 3 phương pháp SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.under_sampling import CondensedNearestNeighbour
from sklearn.model_selection import train_test_split
import numpy as np

def balance_datasets(models_info, method="smote", random_state=42):
    balanced_data = []

    for name, X_train, y_train, X_test, y_test in models_info:
        print(f"\n{name} - Đang cân bằng với phương pháp: {method.upper()}")

        if method == "smote":
            sampler = SMOTE(random_state=random_state)
        elif method == "smoteenn":
            sampler = SMOTEENN(random_state=random_state)
        elif method == "smotecnn":
            # CNN chỉ hỗ trợ under-sampling, nên cần dùng SMOTE trước rồi mới dùng CNN
            X_temp, y_temp = SMOTE(random_state=random_state).fit_resample(X_train, y_train)
            sampler = CondensedNearestNeighbour()
            X_res, y_res = sampler.fit_resample(X_temp, y_temp)

            unique, counts = np.unique(y_res, return_counts=True)
            print(f"Số lượng mẫu sau cân bằng (SMOTECNN): {dict(zip(unique, counts))}")
            balanced_data.append((name, X_res, y_res, X_test, y_test))
            continue
        else:
            raise ValueError("Phương pháp không hợp lệ. Chọn: smote, smoteenn hoặc smotecnn")

        X_res, y_res = sampler.fit_resample(X_train, y_train)
        unique, counts = np.unique(y_res, return_counts=True)
        print(f"Số lượng mẫu sau cân bằng: {dict(zip(unique, counts))}")

        balanced_data.append((name, X_res, y_res, X_test, y_test))

    return balanced_data


## SMOTE

In [ ]:
balanced_models_smote = balance_datasets(models_info, method="smote")

###  Random Forest

In [ ]:
train_and_evaluate_multiple_models(balanced_models_smote, RandomForestClassifier, **model_params)

Tháng 4:
              precision    recall  f1-score   support

     No Rain       0.91      0.98      0.94     46790
        Rain       0.41      0.12      0.19      5289

    accuracy                           0.89     52079
    macro avg       0.66      0.55      0.57     52079
    weighted avg       0.86      0.89      0.87     52079

Tháng 10:
              precision    recall  f1-score   support

     No Rain       0.91      0.92      0.91     57109
        Rain       0.64      0.62      0.63     13583

    accuracy                           0.86     70692
    macro avg       0.78      0.77      0.77     70692
    weighted avg       0.86      0.86      0.86     70692

In [ ]:
!pip install bayesian-optimization scikit-learn pandas

In [ ]:
from bayes_opt import BayesianOptimization
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

In [ ]:
data_dict = {name: (X_res, y_res, X_test, y_test) for name, X_res, y_res, X_test, y_test in balanced_models_smote}

In [ ]:
def make_objective(X_res, y_res):
    def objective(n_estimators, max_depth, min_samples_split, min_samples_leaf, max_features):
        model = RandomForestClassifier(
            n_estimators=int(n_estimators),
            max_depth=int(max_depth),
            min_samples_split=int(min_samples_split),
            min_samples_leaf=int(min_samples_leaf),
            max_features=min(max_features, 0.999),
            class_weight='balanced',
            random_state=42
        )
        score = cross_val_score(model, X_res, y_res, cv=3, scoring="f1").mean()
        return -score  # Maximize F1 → Minimize -F1
    return objective

In [ ]:
pbounds = {
    'n_estimators': (10, 250),
    'max_depth': (1, 50),
    'min_samples_split': (2, 25),
    'min_samples_leaf': (1, 50),
    'max_features': (0.1, 0.999)
}

In [ ]:
# optimizers = {}
# for month in ["Tháng 4", "Tháng 10"]:
#     print(f"\n Đang tối ưu cho {month}...")
#     X_res, y_res, _, _ = data_dict[month]

#     optimizer = BayesianOptimization(
#         f=make_objective(X_res, y_res),
#         pbounds=pbounds,
#         random_state=42
#     )
#     optimizer.maximize(init_points=5, n_iter=15)
#     optimizers[month] = optimizer
#     print(f" Best params for {month}: {optimizer.max}")

Best params for Tháng 4: {'target': -0.6630507661570126, 'params': {'max_depth': 1.9061486915036214, 'max_features': 0.23765197570597946, 'min_samples_leaf': 35.37131602207891, 'min_samples_split': 2.7201556302611016, 'n_estimators': 60.504709358608636}}

Best params for Tháng 10: {'target': -0.793800280961404, 'params': {'max_depth': 2.00864022049432, 'max_features': 0.9719489570936329, 'min_samples_leaf': 41.78968939922066, 'min_samples_split': 6.883799545600351, 'n_estimators': 53.637992129704145}}


In [ ]:
import joblib

best_params = {
    "Tháng 4": {
        'max_depth': int(round(1.9061)),
        'max_features': 0.2376,
        'min_samples_leaf': int(round(35.3713)),
        'min_samples_split': int(round(2.7201)),
        'n_estimators': int(round(60.5047)),
    },
    "Tháng 10": {
        'max_depth': int(round(2.0086)),
        'max_features': 0.9719,
        'min_samples_leaf': int(round(41.7896)),
        'min_samples_split': int(round(6.8838)),
        'n_estimators': int(round(53.638)),
    }
}

for name in ["Tháng 4", "Tháng 10"]:
    print(f"\n=== {name} ===")
    X_res, y_res, X_test, y_test = data_dict[name]
   
    params = best_params[name]
    print(f"Using parameters for {name}: {params}")

    model = RandomForestClassifier(**params, random_state=42)
    model.fit(X_res, y_res)

    model_filename = f"model_{name.replace(' ', '_').lower()}_randomfs.pkl"
    joblib.dump(model, model_filename)

    y_pred = model.predict(X_test)
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

=== Tháng 4 ===

[[36740 10050]
 
 [ 2450  2839]]
                    
                     precision    recall  f1-score   support

           0       0.94      0.79      0.85     46790
           1       0.22      0.54      0.31      5289

    accuracy                           0.76     52079
    macro avg       0.58      0.66      0.58     52079
    weighted avg       0.86      0.76      0.80     52079


=== Tháng 10 ===

[[48709  8400]
 
 [ 3725  9858]]
             
              precision    recall  f1-score   support

           0       0.93      0.85      0.89     57109
           1       0.54      0.73      0.62     13583

    accuracy                           0.83     70692
    macro avg       0.73      0.79      0.75     70692
    weighted avg       0.85      0.83      0.84     70692

### thử điều chỉnh lại ngưỡng phân loại

In [ ]:
# for name in ["Tháng 4", "Tháng 10"]:
#     print(f"\n=== {name} ===")
#     X_res, y_res, X_test, y_test = data_dict[name]
   
#     params = best_params[name]
#     print(f"Using parameters for {name}: {params}")

#     model = RandomForestClassifier(**params, random_state=42)
#     model.fit(X_res, y_res)

#     model_filename = f"model_{name.replace(' ', '_').lower()}_randomfs.pkl"
#     joblib.dump(model, model_filename)

#     y_proba = model.predict_proba(X_test)[:, 1]  # Probability of class 1 (rain)

#     # Adjust classification threshold (e.g., 0.4)
#     threshold = 0.4
#     y_pred_adjusted = (y_proba >= threshold).astype(int)  # Classify as rain (1) if probability >= threshold

#     print(confusion_matrix(y_test, y_pred_adjusted))
#     print(classification_report(y_test, y_pred_adjusted))


=== Tháng 4 ===

 [[25057 21733]

 [ 1389  3900]]
 
              precision    recall  f1-score   support

           0       0.95      0.54      0.68     46790
           1       0.15      0.74      0.25      5289

    accuracy                           0.56     52079
    macro avg       0.55      0.64      0.47     52079
    weighted avg       0.87      0.56      0.64     52079



=== Tháng 10 ===

[[39941 17168]

 [ 1934 11649]]
  
              precision    recall  f1-score   support

           0       0.95      0.70      0.81     57109
           1       0.40      0.86      0.55     13583

    accuracy                           0.73     70692
     macro avg       0.68      0.78      0.68     70692
    weighted avg       0.85      0.73      0.76     70692

### XGBoost

In [ ]:
# train_and_evaluate_multiple_models(balanced_models_smote, XGBClassifier, **xgboost_params)

Tháng 4:
              precision    recall  f1-score   support

     No Rain       0.93      0.84      0.88     46790
        Rain       0.23      0.41      0.29      5289

    accuracy                           0.80     52079
    macro avg       0.58      0.63      0.59     52079
    weighted avg       0.86      0.80      0.82     52079

Tháng 10:
              precision    recall  f1-score   support

     No Rain       0.94      0.86      0.90     57109
        Rain       0.56      0.76      0.64     13583

    accuracy                           0.84     70692
    macro avg       0.75      0.81      0.77     70692
    weighted avg       0.86      0.84      0.85     70692

  - Trước khi áp dụng SMOTE, mô hình có độ recall thấp, dẫn đến khả năng phát hiện mưa kém, thường bỏ sót các trường hợp có mưa;  độ chính xác tổng thể cao, nhưng điều này có thể gây ra sự thiên vị với lớp "Không mưa" (do lớp này chiếm ưu thế), F1 score của lớp mưa thường thấp hoặc ở mức trung bình, cho thấy mô hình chưa thể nhận diện tốt lớp này.
  - Sau khi áp dụng SMOTE, recall tăng lên, giúp mô hình phát hiện mưa tốt hơn và giảm thiểu việc bỏ sót, độ chính xác giảm nhẹ, vì mô hình phải đánh giá công bằng hơn giữa lớp "Mưa" và "Không mưa", nhưng đây là sự cải thiện về mặt cân bằng giữa hai lớp, F1 score của lớp mưa tăng hoặc giữ nguyên, đặc biệt là trong tháng 4, khi mà sự cải thiện trở nên rõ rệt, giúp mô hình đánh giá chính xác hơn đối với lớp mưa.

#### BayesianOptimization

In [ ]:
def make_objective_xgb(X_res, y_res):
    def objective(n_estimators, max_depth, learning_rate, subsample, colsample_bytree):
        model = xgb.XGBClassifier(
            n_estimators=int(n_estimators),
            max_depth=int(max_depth),
            learning_rate=learning_rate,
            subsample=subsample,
            colsample_bytree=colsample_bytree,
            objective='binary:logistic',
            random_state=42
        )
        score = cross_val_score(model, X_res, y_res, cv=3, scoring="f1").mean()
        return -score  # Tối đa hóa F1 → Minimize -F1
    return objective

pbounds = {
    'n_estimators': (50, 500),          
    'max_depth': (3, 15),             
    'learning_rate': (0.01, 0.3),        
    'subsample': (0.5, 1.0),           
    'colsample_bytree': (0.5, 1.0)  
}

optimizers = {}

# for month in ["Tháng 4", "Tháng 10"]:
#     print(f"\nĐang tối ưu hóa cho {month}...")
#     X_res, y_res, _, _ = data_dict[month] 
    
#     optimizer = BayesianOptimization(
#         f=make_objective_xgb(X_res, y_res),
#         pbounds=pbounds,
#         random_state=42
#     )

#     optimizer.maximize(init_points=5, n_iter=15)
#     optimizers[month] = optimizer
#     print(f"Best params for {month}: {optimizer.max}")

Best params for Tháng 4: {'target': -0.7762891357630576, 'params': {'colsample_bytree': 0.9302889641232557, 'learning_rate': 0.03969592497150714, 'max_depth': 3.5260199963228143, 'n_estimators': 157.070149470272, 'subsample': 0.7736235928015441}}

Best params for Tháng 10: {'target': -0.7895895690703282, 'params': {'colsample_bytree': 0.9319409105687462, 'learning_rate': 0.17591251629597351, 'max_depth': 7.995163045267633, 'n_estimators': 132.4638530051047, 'subsample': 0.6407280284500263}}



In [ ]:
# best_params = {
#     "Tháng 4": {
#         'colsample_bytree': 0.9303,
#         'learning_rate': 0.0397,
#         'max_depth': int(round(3.526)),     
#         'n_estimators': int(round(157.07)),
#         'subsample': 0.7736
#     },
#     "Tháng 10": {
#         'colsample_bytree': 0.9319,
#         'learning_rate': 0.1759,
#         'max_depth': int(round(7.995)),
#         'n_estimators': int(round(132.46)),
#         'subsample': 0.6407
#     }
# }

# for name in ["Tháng 4", "Tháng 10"]:
#     print(f"\n=== {name} ===")
#     X_res, y_res, X_test, y_test = data_dict[name]
    
#     params = best_params[name]
#     print(f"Using parameters for {name}: {params}")

#     model = XGBClassifier(
#         **params,
#         use_label_encoder=False,
#         eval_metric='logloss',
#         random_state=42
#     )
#     model.fit(X_res, y_res)

#     model_filename = f"model_{name.replace(' ', '_').lower()}_xgboost.pkl"
#     joblib.dump(model, model_filename)

#     y_pred = model.predict(X_test)
#     print(confusion_matrix(y_test, y_pred))
#     print(classification_report(y_test, y_pred))


=== Tháng 4 ===

[[32146 14644]

 [ 1663  3626]]
      
              precision    recall  f1-score   support

           0       0.95      0.69      0.80     46790
           1       0.20      0.69      0.31      5289

    accuracy                           0.69     52079
    macro avg       0.57      0.69      0.55     52079
    weighted avg       0.87      0.69      0.75     52079


=== Tháng 10 ===

[[50486  6623]

[ 3885  9698]]
             
              precision    recall  f1-score   support

           0       0.93      0.88      0.91     57109
           1       0.59      0.71      0.65     13583

    accuracy                           0.85     70692
    macro avg       0.76      0.80      0.78     70692
    weighted avg       0.86      0.85      0.86     70692

### PNN after SMOTE

In [ ]:
#Fine-tune + train
sigmas = np.logspace(-2, 1, 50)
param_grid = {'sigma': sigmas.tolist()}  # Grid search for sigma values

best_score = 0
best_sigma = 0

# Perform grid search over the values of sigma
print("Fine tuning...")
for name, train_X, train_Y, X_test, y_test in balanced_models_smote:
    X_train, X_val, y_train, y_val = train_test_split(
        train_X, train_Y, test_size=0.2, random_state=42
    )
    print(f"{name}...")
    for sigma in param_grid['sigma']:
        pnn_batch = PNN(sigma=sigma, batch_size=2048)
        pnn_batch.fit(X_train, y_train)
        y_pred = pnn_batch.predict(X_val)
        score = classification_report(y_val, y_pred, zero_division=0, output_dict=True)['macro avg']['f1-score']
    
        if score > best_score:
            best_score = score
            best_sigma = sigma

    print(f"Best sigma: {best_sigma} with f1: {best_score}")

# Final evaluation with the best sigma
print("Train with best param...")
for name, X_train, y_train, X_test, y_test in balanced_models_smote:
    print(f"{name}...")
    print(X_train.shape)
    pnn_batch = PNN(sigma=best_sigma, batch_size=1000)
    pnn_batch.fit(X_train, y_train)
    y_pred = pnn_batch.predict(X_test)

    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    # Optionally, plot the confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["No Rain", "Rain"], yticklabels=["No Rain", "Rain"])
    plt.title("Confusion Matrix")
    plt.show()



### SCG

In [ ]:
param_grid = {'hidden_dim': [10, 20, 50, 100, 150, 17, 25, 34, 50, 68, 85]}

for name, X_trval, y_trval, X_test, y_test in balanced_models_smote:
    print(f"\n=== Dataset: {name} ===")
    # split train+val from test
    X_train, X_val, y_train, y_val = train_test_split(
        X_trval, y_trval, test_size=0.2, random_state=42
    )
    # one-hot encode y_train
    y_arr = y_train.to_numpy().reshape(-1, 1)
    encoder = OneHotEncoder(sparse=False, categories='auto')
    Y_train_oh = encoder.fit_transform(y_arr.reshape(-1,1))
    y_arv = y_val.to_numpy().reshape(-1, 1)
    Y_val_oh   = encoder.transform(y_arv.reshape(-1,1))
    # final test encoding not needed for training
    
    # scale assumed done before, X_trval and X_test are already scaled
    
    input_dim  = X_train.shape[1]
    output_dim = Y_train_oh.shape[1]
    
    best_h, best_loss = None, np.inf
    for h in param_grid['hidden_dim']:
        # init params
        param_size  = h*input_dim + h + output_dim*h + output_dim
        init_p = 0.01 * np.random.randn(param_size)
        # train SCG
        lg = loss_grad_factory(X_train, Y_train_oh, input_dim, h, output_dim)
        opt_p, _ = trainscg(lg, init_p, epochs=8)
        # predict on val
        W1, b1, W2, b2 = unpack(opt_p, input_dim, h, output_dim)
        Z1_val = X_val.dot(W1.T) + b1
        A1_val = np.tanh(Z1_val)
        Z2_val = A1_val.dot(W2.T) + b2
        A2_val = softmax(Z2_val)
        val_loss = -np.mean(np.sum(Y_val_oh * np.log(A2_val + 1e-12), axis=1))
        print(f" hidden_dim={h:<3} → val loss = {val_loss:.6f}")
        if val_loss < best_loss:
            best_loss, best_h = val_loss, h
    
    print(f"⇒ Best hidden_dim = {best_h}, validation loss = {best_loss:.6f}")
    
    # retrain on full train+val
    X_full = np.vstack([X_train, X_val])
    y_full = np.hstack([y_train, y_val])
    Y_full_oh = encoder.fit_transform(y_full.reshape(-1,1))
    param_size  = best_h*input_dim + best_h + output_dim*best_h + output_dim
    init_p_full = 0.01 * np.random.randn(param_size)
    lg_full = loss_grad_factory(X_full, Y_full_oh, input_dim, best_h, output_dim)
    opt_p_full, loss_hist = trainscg(lg_full, init_p_full, epochs=8)
    
    # evaluate on test
    W1f, b1f, W2f, b2f = unpack(opt_p_full, input_dim, best_h, output_dim)
    A1_test = np.tanh(X_test.dot(W1f.T) + b1f)
    probs_test = softmax(A1_test.dot(W2f.T) + b2f)
    preds_test = np.argmax(probs_test, axis=1)
    
    print("Test classification report:")
    print(classification_report(y_test, preds_test))
    cm = confusion_matrix(y_test, preds_test)
    print("Test confusion matrix:\n", cm)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title(f"{name} – hidden_dim={best_h}")
    plt.show()

## SMOTE CNN

In [ ]:
# # Dùng SMOTE + CNN
# balanced_models_smotecnn = balance_datasets(models_info, method="smotecnn")

## SMOTE ENN

In [ ]:
# # Dùng SMOTE + ENN
balanced_models_smoteenn = balance_datasets(models_info, method="smoteenn")

In [ ]:
#Fine-tune + train
sigmas = np.logspace(-2, 1, 50)
param_grid = {'sigma': sigmas.tolist()}  # Grid search for sigma values

best_score = 0
best_sigma = 0

# Perform grid search over the values of sigma
print("Fine tuning...")
for name, train_X, train_Y, X_test, y_test in balanced_models_smoteenn:
    X_train, X_val, y_train, y_val = train_test_split(
        train_X, train_Y, test_size=0.2, random_state=42
    )
    print(f"{name}...")
    for sigma in param_grid['sigma']:
        pnn_batch = PNN(sigma=sigma, batch_size=2048)
        pnn_batch.fit(X_train, y_train)
        y_pred = pnn_batch.predict(X_val)
        score = classification_report(y_val, y_pred, zero_division=0, output_dict=True)['macro avg']['f1-score']
    
        if score > best_score:
            best_score = score
            best_sigma = sigma

    print(f"Best sigma: {best_sigma} with f1: {best_score}")

# Final evaluation with the best sigma
print("Train with best param...")
for name, X_train, y_train, X_test, y_test in balanced_models_smoteenn:
    print(f"{name}...")
    print(X_train.shape)
    pnn_batch = PNN(sigma=best_sigma, batch_size=1000)
    pnn_batch.fit(X_train, y_train)
    y_pred = pnn_batch.predict(X_test)

    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    # Optionally, plot the confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["No Rain", "Rain"], yticklabels=["No Rain", "Rain"])
    plt.title("Confusion Matrix")
    plt.show()

In [ ]:
param_grid = {'hidden_dim': [10, 20, 50, 100, 150, 17, 25, 34, 50, 68, 85]}

for name, X_trval, y_trval, X_test, y_test in balanced_models_smoteenn:
    print(f"\n=== Dataset: {name} ===")
    # split train+val from test
    X_train, X_val, y_train, y_val = train_test_split(
        X_trval, y_trval, test_size=0.2, random_state=42
    )
    # one-hot encode y_train
    y_arr = y_train.to_numpy().reshape(-1, 1)
    encoder = OneHotEncoder(sparse=False, categories='auto')
    Y_train_oh = encoder.fit_transform(y_arr.reshape(-1,1))
    y_arv = y_val.to_numpy().reshape(-1, 1)
    Y_val_oh   = encoder.transform(y_arv.reshape(-1,1))
    # final test encoding not needed for training
    
    # scale assumed done before, X_trval and X_test are already scaled
    
    input_dim  = X_train.shape[1]
    output_dim = Y_train_oh.shape[1]
    
    best_h, best_loss = None, np.inf
    for h in param_grid['hidden_dim']:
        # init params
        param_size  = h*input_dim + h + output_dim*h + output_dim
        init_p = 0.01 * np.random.randn(param_size)
        # train SCG
        lg = loss_grad_factory(X_train, Y_train_oh, input_dim, h, output_dim)
        opt_p, _ = trainscg(lg, init_p, epochs=8)
        # predict on val
        W1, b1, W2, b2 = unpack(opt_p, input_dim, h, output_dim)
        Z1_val = X_val.dot(W1.T) + b1
        A1_val = np.tanh(Z1_val)
        Z2_val = A1_val.dot(W2.T) + b2
        A2_val = softmax(Z2_val)
        val_loss = -np.mean(np.sum(Y_val_oh * np.log(A2_val + 1e-12), axis=1))
        print(f" hidden_dim={h:<3} → val loss = {val_loss:.6f}")
        if val_loss < best_loss:
            best_loss, best_h = val_loss, h
    
    print(f"⇒ Best hidden_dim = {best_h}, validation loss = {best_loss:.6f}")
    
    # retrain on full train+val
    X_full = np.vstack([X_train, X_val])
    y_full = np.hstack([y_train, y_val])
    Y_full_oh = encoder.fit_transform(y_full.reshape(-1,1))
    param_size  = best_h*input_dim + best_h + output_dim*best_h + output_dim
    init_p_full = 0.01 * np.random.randn(param_size)
    lg_full = loss_grad_factory(X_full, Y_full_oh, input_dim, best_h, output_dim)
    opt_p_full, loss_hist = trainscg(lg_full, init_p_full, epochs=8)
    
    # evaluate on test
    W1f, b1f, W2f, b2f = unpack(opt_p_full, input_dim, best_h, output_dim)
    A1_test = np.tanh(X_test.dot(W1f.T) + b1f)
    probs_test = softmax(A1_test.dot(W2f.T) + b2f)
    preds_test = np.argmax(probs_test, axis=1)
    
    print("Test classification report:")
    print(classification_report(y_test, preds_test))
    cm = confusion_matrix(y_test, preds_test)
    print("Test confusion matrix:\n", cm)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title(f"{name} – hidden_dim={best_h}")
    plt.show()